## IQ Seminar - Oct 13, 2018
 
Master of Financial Engineering Program, Baruch College   
  
<img src="http://mfe.baruch.cuny.edu/wp-content/uploads/2014/09/BCCUNYstacked_BLK.jpg" align = "left" width=160>  

Questions from Dan, for MS/CS phone etc.

# Definition Questions 

## 1. Hash Table

### 1.1 What is hash tabe?

#### Associative array

A hash table is a data structure used to implement an associative array that can map keys to values. A hash table uses a hash function to compute an index into an array of buckets, from which the desired value can be found.

#### Collisions

The hash function will assign each key to a unique bucket, but it is possible that two keys will generate an identical hash causing both keys to point to the same bucket. Instead, most hash table designs assume that hash collisions—different keys that are assigned by the hash function to the same bucket—will occur and must be accommodated in some way. (Chaining, open addressing)

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Hash_table_5_0_1_1_1_1_0_SP.svg/380px-Hash_table_5_0_1_1_1_1_0_SP.svg.png'>

#### Average cost / complexity

In a well-dimensioned hash table, the average cost for each lookup is independent of the number of elements stored in the table. Many hash table designs also allow arbitrary insertions and deletions of key-value pairs, at constant average cost per operation.

### 1.2 How to implement it?

In [23]:
class HashTable:
    def __init__(self):
        self.size = 10
        self.slots = [None] * self.size
        self.data = [None] * self.size
        
    def put(self,key,data):
        hashvalue = self.hashfunction(key,len(self.slots))

        if self.slots[hashvalue] == None:
            self.slots[hashvalue] = key
            self.data[hashvalue] = data
        else:
            if self.slots[hashvalue] == key:
                self.data[hashvalue] = data  #replace
            else:
                nextslot = self.rehash(hashvalue,self.size)
                while self.slots[nextslot] != None and self.slots[nextslot] != key:
                    nextslot = self.rehash(nextslot,self.size)
                if self.slots[nextslot] == None:
                    self.slots[nextslot]=key
                    self.data[nextslot]=data
                else:
                    self.data[nextslot] = data #replace

    def hashfunction(self,key,size):
        return key%size

    def rehash(self,oldhash,size):
        return (oldhash+1)%size
    
    def get(self,key):
        startslot = self.hashfunction(key,len(self.slots))

        data = None
        stop = False
        found = False
        position = startslot
        while self.slots[position] != None and not found and not stop:
            if self.slots[position] == key:
                found = True
                data = self.data[position]
            else:
                position=self.rehash(position,len(self.slots))
                if position == startslot:
                    stop = True
        return data

    def __getitem__(self,key):
        return self.get(key)

    def __setitem__(self,key,data):
        self.put(key,data)

In [24]:
H=HashTable()
H[54]="cat"
H[26]="dog"
H[93]="lion"
H[17]="tiger"
H[77]="bird"
H[31]="cow"
H[44]="goat"
H[55]="pig"
H[20]="chicken"
H.slots

[20, 31, None, 93, 54, 44, 26, 17, 77, 55]

In [25]:
H.data

['chicken', 'cow', None, 'lion', 'cat', 'goat', 'dog', 'tiger', 'bird', 'pig']

## 2. What design pattern do you know?

- Design patterns are optimized, reusable solutions to the programming problems that we encounter every day. 
- A design pattern is not a class or a library that we can simply plug into our system; it's much more than that. It is a template that has to be implemented in the correct situation.

### 2.1 Singleton

Singleton pattern provides a mechanism to limit the number of the instances of the class to one. Thus the same object is always shared by different parts of the code.

<img src='http://www.noesispoint.com/img/DesignPattern/Singleton-Design-Pattern.jpg'>

In [8]:
class Singleton(Exception):
    '''
    Singleton exception(error) class

    Because Python has no private constructors we have to find some other way to prevent instantiations. 
    Our approach is to raise an exception if the Singleton object is already instantiated 
    (private class attribute __single is other than None). The exception object is the Singleton instance
    '''
    __single = None
    def __init__( self ):
        if Singleton.__single:
            raise Singleton.__single
        Singleton.__single = self 

# Creation
        
def Handle( x = Singleton ):

    try:
        single = x()
    except Singleton as s:
        single = s
    return single 

# Subclasses

'''
class Singleton has subclasses Child and Junior as shown. Because there exists an is-a relationship between 
base class and a subclass there can be only one instance of the whole hierarchy. Singleton or Junior instance 
can be created with Handle function. The constructor of the Child class takes one - explicit - argument so the 
instantiation must be done with the direct constructor call or a specialized handle function. 
'''

class Child( Singleton ):
    def __init__( self, name ):
        Singleton.__init__( self )
        self.__name = name
    def name( self ):
        return self.__name

class Junior( Singleton ):
    pass

In [9]:
child = Child( 'Monty' )
junior = Handle( Junior )
junior.name()

'Monty'

In [10]:
single = Handle()
single

__main__.Child('Monty')

In [11]:
child = Child('ABC') # Error

Child: Monty

### 2.2 Chain of Responsibility

- The chain of Responsibility pattern is to create a system that can serve different requests in a hierarchical manner. 

- In other words if an object that is a part of the system does not know how to respond to the given request it passes it along the object tree. Each object along the route of the request can take the responsibility and serve the request.

<img src='http://i.ytimg.com/vi/BsuhLJb6vo0/maxresdefault.jpg'>

In [12]:
class Event:
    def __init__( self, name ):
        self.name = name

class Widget:
    '''
    The idea behind the Command Dispatch pattern is to check at runtime whether the object has a proper method or not.
    '''
    def __init__( self, parent = None ):
        self.__parent = parent
    def Handle( self, event ):
        handler = 'Handle_' + event.name
        if hasattr( self, handler ):
            method = getattr( self, handler )
            method( event )
        elif self.__parent:
            self.__parent.Handle( event )
        elif hasattr( self, 'HandleDefault' ):
            self.HandleDefault( event )    

When an event object is passed to Handle method, one of the four things may happen:

- Each event has an associated name. If widget has a corresponding method (named Handle_eventname ), it is executed
- If the object is not the last one in the chain, it passes the event object to its parent
- If the object has a default handler for all kinds of events (HandleDefault method) it is invoked if the object is the last possible handler. Because HandleDefault should always be available only on the end of the chain, it is not necessary or even sensible to have it in the interface of the base class.
- The object dismisses the event if it can neither handle or forward it.

In [14]:
class MainWindow( Widget ):
    def Handle_close( self, event ):
        print('MainWindow: ' + event.name)
    def HandleDefault( self, event ):
        print('Default: ' + event.name)
        
class SendDialog( Widget ):
    def Handle_paint( self, event ):
        print('SendDialog: ' + event.name)

class MsgText( Widget ):
    def Handle_down( self, event ):
        print('MsgText: ' + event.name)

In [15]:
# Define hierarchi

mw = MainWindow()
sd = SendDialog( mw )
msg = MsgText( sd )

# Define event
edown = Event( 'down' )
epaint = Event( 'paint' )
eodd = Event( 'odd' )

In [16]:
msg.Handle( edown )

MsgText: down


In [17]:
msg.Handle( epaint )

SendDialog: paint


In [18]:
msg.Handle( eodd )

Default: odd


### 2.3 Proxy

- The Proxy pattern is used when an object has to be shielded from its clients. 

- There may be a number of reasons for this: reference counting, different levels of access rights, lazy evaluation of the state of the object and so on.

- The client does not need to know that it is not accessing the real object (Subject) directly. In other words the proxy substitutes for the real thing.

<img src='https://sourcemaking.com/files/v2/content/patterns/Proxy_example1-2x.png'>

In [68]:
# Real subject

class RGB:
    def __init__( self, red, green, blue ):
        self.__red = red
        self.__green = green
        self.__blue = blue
    def Red( self ):
        return self.__red
    def Green( self ):
        return self.__green
    def Blue( self ):
        return self.__blue 


In [74]:
# Proxy base class, return the same subject itself

class Proxy:
    def __init__( self, subject ):
        self.__subject = subject
    def __getattr__( self, name ):
        return getattr( self.__subject, name )
    def html_str(self):
        return 'rgb(%i,%i,%i)'%(self.Red(), self.Green(), self.Blue())
    
# Hacker

class NoBlueProxy( Proxy ):
    def Blue( self ):
        return 0

In [75]:
from IPython.display import HTML
rgb = RGB( 100, 192, 240 )

In [76]:
proxy = Proxy( rgb )
print(proxy.html_str())
HTML(cl.to_html({'RGB':[proxy.html_str()]}))

rgb(100,192,240)


In [78]:
noblue = NoBlueProxy( rgb )
print(noblue.html_str())
HTML(cl.to_html({'RGB':[noblue.html_str()]}))

rgb(100,192,0)


### 2.4 Bridge Pattern

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/c/cf/Bridge_UML_class_diagram.svg/500px-Bridge_UML_class_diagram.svg.png'>

In [99]:
class AbstractInterface:
    """ Target interface.
    This is the target interface, that clients use.
    """
    def someFunctionality(self):
        raise NotImplemented()


class Bridge(AbstractInterface):
    """ Bridge class.
    
    This class forms a bridge between the target
    interface and background implementation.
    """
    def __init__(self):
        self.__implementation = None


class UseCase1(Bridge):
    """ Variant of the target interface.
    This is a variant of the target Abstract interface.
    It can do something little differently and it can
    also use various background implementations through
    the bridge.
    """
    def __init__(self, implementation):
        self.__implementation = implementation
    def someFunctionality(self):
        print ("UseCase1: ",)
        self.__implementation.anotherFunctionality()

class UseCase2(Bridge):
    def __init__(self, implementation):
        self.__implementation = implementation
    def someFunctionality(self):
        print ("UseCase2: ",)
        self.__implementation.anotherFunctionality()

class ImplementationInterface:
    """ Interface for the background implementation.
    This class defines how the Bridge communicates
    with various background implementations.
    """
    def anotherFunctionality(self):
        raise NotImplemented

class Linux(ImplementationInterface):
    """ Concrete background implementation.
    A variant of background implementation, in this
    case for Linux!
    """
    def anotherFunctionality(self):
        print ("Linux!")

class Windows(ImplementationInterface):
    def anotherFunctionality(self):
        print ("Windows.")

def main():
    linux = Linux()
    windows = Windows()

    # Couple of variants under a couple
    # of operating systems.
    useCase = UseCase1(linux)
    useCase.someFunctionality()

    useCase = UseCase1(windows)
    useCase.someFunctionality()

    useCase = UseCase2(linux)
    useCase.someFunctionality()

    useCase = UseCase2(windows)
    useCase.someFunctionality()

if __name__ == "__main__":
    main()

UseCase1: 
Linux!
UseCase1: 
Windows.
UseCase2: 
Linux!
UseCase2: 
Windows.


## 3. Virtual Function

### 3.1 What's virtual function

- A virtual function (virtual method) is a function whose behavior can be overridden within an inheriting class by a function with the same signature. 

- This concept is an important part of the polymorphism portion of object-oriented programming (OOP).

### 3.2 Why do we need it

In object-oriented programming, when a derived class inherits from a base class, an object of the derived class may be referred to via a pointer or reference of the base class type instead of the derived class type. If there are base class methods overridden by the derived class, the method actually called by such a reference or pointer can be bound either 'early' (by the compiler), according to the declared type of the pointer or reference, or 'late' (i.e., by the runtime system of the language), according to the actual type of the object referred to.

<img src='http://www.studyvilla.com/Images/poly.jpg'>

```cpp
class Animal {
    void /*non-virtual*/ move(void) { 
        std::cout << "This animal moves in some way" << std::endl; 
    }
    virtual void eat(void) {}
};

// The class "Animal" may possess a definition for eat() if desired.
class Wolf : public Animal {
    // The non virtual function move() is inherited but not overridden
    void eat(void) { 
        std::cout << "Llamas eat grass!" << std::endl; 
    }
};
```

### 3.3 Abstract Class (ABC) / Pure Virtual

A pure virtual function or pure virtual method is a virtual function that is required to be implemented by a derived class if the derived class is not abstract. Classes containing pure virtual methods are termed "abstract" and $\textbf{they cannot be instantiated directly}$. A subclass of an abstract class can only be instantiated directly if $\textbf{all inherited pure virtual methods have been implemented}$ by that class or a parent class. Pure virtual methods typically have a declaration (signature) and no definition (implementation).

Although pure virtual methods typically have no implementation in the class that declares them, pure virtual methods in C++ are permitted to contain an implementation in their declaring class, providing default behaviour that a derived class can delegate to, if appropriate.

### 3.4 Virtual Destructor

- Object-oriented languages typically manage memory allocation and de-allocation automatically when objects are created and destroyed. However, some object-oriented languages allow a custom destructor method to be implemented, if desired. (Python, Java)

- If the language in question uses automatic memory management, the custom destructor (generally called a finalizer in this context) that is called is certain to be the appropriate one for the object in question. For example, if an object of type Wolf that inherits Animal is created, and both have custom destructors, the one called will be the one declared in Wolf.

In [12]:
class FooType(object):
    def __init__(self, id):
        self.id = id
        print (self.id, 'born')

    def __del__(self):
        print (self.id, 'died')

def test():
    ft3 = FooType(3)
    
test()

3 born
3 died


In [97]:
class Base(object):
    def __init__(self, id):
        self.id = id
        print (self.id, 'base born')

    def __del__(self):
        print (self.id, 'base died')
        
class Derived(Base):
    def __del__(self):
        print (self.id, 'derived died')

def test2():
    derived = Derived(1)
    
test2()

1 base born
1 derived died


- In manual memory management contexts, the situation can be more complex. If an object of type Wolf is created but pointed to by an Animal pointer, and it is this Animal pointer type that is deleted, the destructor called may actually be the one defined for Animal and not the one for Wolf, unless the destructor is virtual. This is particularly the case with C++, where the behaviour is a common source of programming errors.

```cpp
int main(int argc, char const *argv[])
{
    Animal* AnimalPtr = new Wolf;
    
    delete AnimalPtr; 
    // if the destructor in Animal class is not virtual, then Animal destructor is called, which might be wrong.
    // if the destructor in Animal class is virtual, then Wolf and Animal destructor are both called.
    
	return 0;
}

```

## 4. Memory Leak

### 4.1 Definition

In computer science, a memory leak is a type of resource leak that occurs when a computer program incorrectly manages memory allocations in such a way that memory which is no longer needed is not released. 

In object-oriented programming, a memory leak may happen when an object is stored in memory but cannot be accessed by the running code. A memory leak has symptoms similar to a number of other problems and generally can only be diagnosed by a programmer with access to the program's source code.

<img src='http://www.programcreek.com/wp-content/uploads/2013/10/where-is-memory-leak-650x400.jpeg'>

### 4.2 Consequences

- A memory leak can diminish the performance of the computer by reducing the amount of available memory. All or part of the system or device stops working correctly, the application fails, or the system slows down vastly due to thrashing.

- Memory leaks may not be serious or even detectable by normal means. In modern operating systems, normal memory used by an application is released when the application terminates. This means that a memory leak in a program that only runs for a short time may not be noticed and is rarely serious.

### 4.3 Example

Pseudocode:

```cpp
while (button_pressed == true):
    floor_number* ptr = new floor_number; // which will be used to remember the floor number
    
    if (curr_floor == target_floor) // Are we already on the target floor?
        return; // if so, nothing to do
    else {
        while (lift_isIdle == false) {} // wait until the lift is idle
        goto_requiredFloor();
        delete ptr; // Release the memory we used to remember the floor number
    }
```

A simple example in C:

```c
#include <stdlib.h>

void function_which_allocates(void) {
    /* allocate an array of 45 floats */
    float *a = malloc(sizeof(float) * 45);

    /* additional code making use of 'a' */

    /* return to main, having forgotten to free the memory we malloc'd */
}

int main(void) {
    function_which_allocates();

    /* the pointer 'a' no longer exists, and therefore cannot be freed,
     but the memory is still allocated. a leak has occurred. */
}
```